<!--NAVIGATION-->
< [Output widgets: leveraging Jupyter's display system](04.01-more-on-output-widget.ipynb) | [Contents](00.00-index.ipynb) | [*OPTIONAL* Three approaches to events](05.01-OPTIONAL-Widget_Events_2.ipynb) >

# Widgets Events

In [27]:
import ipywidgets as widgets
from IPython.display import display

## Traitlets events

Every widget class is an `HasTraits` class, which means they benefit from the Traitlets API concerning the validation and observation of properties (see https://traitlets.readthedocs.io/en/stable/using_traitlets.html#using-traitlets).

### Trait class example: properties **validation** and **observation**

Traitlets are validated by **type** and **value**:

In [28]:
from traitlets import HasTraits, Unicode, Int, TraitError, validate

class Identity(HasTraits):

    username = Unicode()
    age = Int()

    @validate('age')
    def _validate_age(self, proposal):
        age = proposal['value']

        if age < 0:
            raise TraitError('age can not be negative')

        if age > 115:
            raise TraitError('this is too old to be true')

        return age

In [29]:
jane = Identity(username='Jane Doe', age=25)
jane.age

25

In [30]:
# jane.age = 130  # Breaks!

In [31]:
jane.age = 32

Every `HasTraits` class has an `observe` method which allows observing properties changes. You can assign a Python callback function that will be called when a property changes.

The callback handler passed to observe will be called with one change argument. The change object holds at least a `type` key and a `name` key, corresponding respectively to the type of notification and the name of the attribute that triggered the notification.

Other keys may be passed depending on the value of `type`. In the case where type is `change`, we also have the following keys:

- `owner` : the HasTraits instance
- `old` : the old value of the modified trait attribute
- `new` : the new value of the modified trait attribute
- `name` : the name of the modified trait attribute.

In [32]:
HasTraits.observe?

Signature: HasTraits.observe(self, handler, names=traitlets.All, type='change')
Docstring:
Setup a handler to be called when a trait changes.

This is used to setup dynamic notifications of trait changes.

Parameters
----------
handler : callable
    A callable that is called when a trait changes. Its
    signature should be ``handler(change)``, where ``change`` is a
    dictionary. The change dictionary at least holds a 'type' key.
    * ``type``: the type of notification.
    Other keys may be passed depending on the value of 'type'. In the
    case where type is 'change', we also have the following keys:
    * ``owner`` : the HasTraits instance
    * ``old`` : the old value of the modified trait attribute
    * ``new`` : the new value of the modified trait attribute
    * ``name`` : the name of the modified trait attribute.
names : list, str, All
    If names is All, the handler will apply to all traits.  If a list
    of str, handler will apply to all names in the list.  If a
    s

In [33]:
# We use an output widget here for capturing the print calls and showing them at the right place in the Notebook
output = widgets.Output()

@output.capture()
def print_change(change):
    print(change)

# Observe jane.age changes, and print them
jane.observe(print_change, 'age')

output

Output()

In [34]:
# changes here affect ABOVE cell's output!

jane.age = 76

### Registering callbacks to trait changes in the kernel

Since `Widget` classes inherit from `HasTraits`, you can register handlers to the change events whenever the model gets updates from the front-end.

In [35]:
widgets.Widget.observe?

Signature: widgets.Widget.observe(self, handler, names=traitlets.All, type='change')
Docstring:
Setup a handler to be called when a trait changes.

This is used to setup dynamic notifications of trait changes.

Parameters
----------
handler : callable
    A callable that is called when a trait changes. Its
    signature should be ``handler(change)``, where ``change`` is a
    dictionary. The change dictionary at least holds a 'type' key.
    * ``type``: the type of notification.
    Other keys may be passed depending on the value of 'type'. In the
    case where type is 'change', we also have the following keys:
    * ``owner`` : the HasTraits instance
    * ``old`` : the old value of the modified trait attribute
    * ``new`` : the new value of the modified trait attribute
    * ``name`` : the name of the modified trait attribute.
names : list, str, All
    If names is All, the handler will apply to all traits.  If a list
    of str, handler will apply to all names in the list.  If a


The following uses the traits/callbacks to adjust label!

In [36]:
caption = widgets.Label(value='Start moving the slider!')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    sign = 'negative' if change.new < 0 else 'positive'
    caption.value = f'The slider value is {sign}'

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='Start moving the slider!')

IntSlider(value=1, description='Slider', max=5, min=-5)

### Callback signatures

Mentioned in the doc string, the callback registered must have the signature `handler(change)` where `change` is a dictionary holding the information about the change. 

Using this method, an example of how to output an `IntSlider`'s value as it is changed can be seen below.

In [37]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)

def on_value_change(change):
    output2.clear_output(wait=True)

    old = change['old']
    new = change['new']

    with output2:
        print(f'The value was {old} and is now {new}')

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

### Why `observe` instead of `link`?

Using `link` is great if no transformation of the values is needed. `observe` is useful if some kind of calculation needs to be done with the values or if the values that are related have different types.

The example below converts between Celsius and Farhenheit. As written, changing the temperature in Celcius will update the temperature in Farenheit, but not the other way around. You will add that as an exercise.

In [38]:
def C_to_F(temp):
    return 1.8 * temp + 32

def F_to_C(temp):
    return (temp -32) / 1.8

degree_C = widgets.FloatText(description='Temp $^\circ$C', value=0)
degree_F = widgets.FloatText(description='Temp $^\circ$F', value=C_to_F(degree_C.value))

def on_C_change(change):
    degree_F.value = C_to_F(change['new'])
    
degree_C.observe(on_C_change, names='value')

display(degree_C, degree_F)

FloatText(value=0.0, description='Temp $^\\circ$C')

FloatText(value=32.0, description='Temp $^\\circ$F')

### Exercise

Add a callback that is called when `degree_F` is changed. An outline of the callback function is below. Fill it in, and make `degree_F` `observe` call `on_F_change` if the `value` changes.

In [39]:
def on_F_change(change):
    degree_C.value = F_to_C(change['new'])
    
degree_F.observe(on_F_change, names='value')

## Advanced Widget Linking

In an earlier notebook you used `link` to link the value of one widget to another. 

There are a couple of other linking methods that offer more flexibility:

+ `dlink` is a *directional* link; updates happen in one direction but not the other.
+ `jslink` and `jsdlink` do the linking in the front end (i.e. in JavaScript without any communication to Python). 

### Linking traitlets attributes in the kernel (ie. in Python)

The first method is to use the `link` and `dlink`. This only works if we are interacting with a live kernel.

In [40]:
caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')

display(caption, sliders1, slider2)

l = widgets.link((sliders1, 'value'), (slider2, 'value'))


Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [41]:
caption = widgets.HTML(value='Changes in source values are reflected in target1, but changes in target1 do not affect source')
source, target1 = widgets.IntSlider(description='Source'),\
                  widgets.IntSlider(description='Target 1')

display(caption, source, target1)

dl = widgets.dlink((source, 'value'), (target1, 'value'))


HTML(value='Changes in source values are reflected in target1, but changes in target1 do not affect source')

IntSlider(value=0, description='Source')

IntSlider(value=0, description='Target 1')

Links can be broken by calling `unlink`.

In [42]:
l.unlink()
dl.unlink()

Function `widgets.jslink` returns a `Link` widget. The link can be broken by calling the `unlink` method.

### Linking widgets attributes from the client side

You can also directly link widget attributes in the browser using the link widgets, in either a unidirectional or a bidirectional fashion.

Javascript links persist when embedding widgets in html web pages without a kernel.

In [43]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
range1, range2 = widgets.IntSlider(description='Range 1'),\
                 widgets.IntSlider(description='Range 2')

display(caption, range1, range2)

l = widgets.jslink((range1, 'value'), (range2, 'value'))

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=0, description='Range 1')

IntSlider(value=0, description='Range 2')

In [44]:
caption = widgets.Label(value='Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                              widgets.IntSlider(description='Target range 1')

display(caption, source_range, target_range1)

dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))

Label(value='Changes in source_range values are reflected in target_range1')

IntSlider(value=0, description='Source range')

IntSlider(value=0, description='Target range 1')

The links can be broken by calling the `unlink` method.

In [45]:
l.unlink()
dl.unlink()

### The difference between linking in the kernel and linking in the client

Linking in the kernel means linking via python. If two sliders are linked in the kernel, when one slider is changed the browser sends a message to the kernel (python in this case) updating the changed slider, the link widget in the kernel then propagates the change to the other slider object in the kernel, and then the other slider's kernel object sends a message to the browser to update the other slider's views in the browser. If the kernel is not running (as in a static web page), then the controls will not be linked.

Linking using jslink (i.e., on the browser side) means contructing the link in Javascript. When one slider is changed, Javascript running in the browser changes the value of the other slider in the browser, without needing to communicate with the kernel at all. If the sliders are attached to kernel objects, each slider will update their kernel-side objects independently.

To see the difference between the two, go to the [static version of this page in the ipywidgets documentation](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html) and try out the sliders near the bottom. The ones linked in the kernel with `link` and `dlink` are no longer linked, but the ones linked in the browser with `jslink` and `jsdlink` are still linked.

### Continuous vs delayed updates

Some widgets offer a choice with their `continuous_update` attribute between continually updating values or only updating values when a user submits the value (for example, by pressing Enter or navigating away from the control). In the next example, we see the "Delayed" controls only transmit their value after the user finishes dragging the slider or submitting the textbox. The "Continuous" controls continually transmit their values as they are changed. Try typing a two-digit number into each of the text boxes, or dragging each of the sliders, to see the difference.

In [49]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

Sliders, `Text`, and `Textarea` controls default to `continuous_update=True`. `IntText` and other text boxes for entering integer or float numbers default to `continuous_update=False` (since often you'll want to type an entire number before submitting the value by pressing enter or navigating out of the box).

## Special events

Some widgets like the `Button` have special events on which you can hook Python callbacks.

The `Button` is not used to represent a data type.  Instead the button widget is used to handle mouse clicks.  The `on_click` method of the `Button` can be used to register function to be called when the button is clicked.  The doc string of the `on_click` can be seen below.

In [47]:
widgets.Button.on_click?

Signature: widgets.Button.on_click(self, callback, remove=False)
Docstring:
Register a callback to execute when the button is clicked.

The callback will be called with one argument, the clicked button
widget instance.

Parameters
----------
remove: bool (optional)
    Set to true to remove the callback from the list of callbacks.
File:      d:\mdt\miniconda3\envs\widgets-tutorial\lib\site-packages\ipywidgets\widgets\widget_button.py
Type:      function


### Example

Since button clicks are stateless, they are transmitted from the front-end to the back-end using custom messages.  By using the `on_click` method, a button that prints a message when it has been clicked is shown below. To capture `print`s (or any other kind of output including errors) and ensure it is displayed, be sure to send it to an `Output` widget (or put the information you want to display into an `HTML` widget).

In [48]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

@output.capture()
def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

<!--NAVIGATION-->
< [Output widgets: leveraging Jupyter's display system](04.01-more-on-output-widget.ipynb) | [Contents](00.00-index.ipynb) | [*OPTIONAL* Three approaches to events](05.01-OPTIONAL-Widget_Events_2.ipynb) >